Optimisation problem seeks to solve the optimum amount of each cargo type that should be stored in each csection. Given weight and volume constraints of the plane. PRofit should be maximised

In [5]:
from pyomo.environ import *

In [6]:
model       = AbstractModel()
model.i     = Set()
model.j     = Set()
model.planeWeight = Param(model.j,mutable=True)
model.planeSpace  = Param(model.j,mutable=True)
model.CargoW      = Param(model.i,mutable=True)
model.CargoVol    = Param(model.i,mutable=True)
model.CargoProfit = Param(model.i,mutable=True)

def xbounds(model,i,j):
    return (0,model.CargoW[i])
model.x = Var(model.i,model.j, bounds=xbounds,domain=NonNegativeReals)
#model.x = Var(model.i,model.j, bounds=xbounds,domain=NonNegativeIntegers)

#model.balance = Var( bounds=(0,100),domain=NonNegativeReals)

def con1_rule(model,i):
    return sum( model.x[i,j] for j in model.j) <= model.CargoW[i]
model.con1 = Constraint(model.i,rule=con1_rule)

def con2_rule(model,j):
    return sum( model.x[i,j] for i in model.i) <= model.planeWeight[j]
model.con2 = Constraint(model.j,rule=con2_rule)

def con3_rule(model,j):
    return sum( model.x[i,j]*model.CargoVol[i] for i in model.i) <= model.planeSpace[j]
model.con3 = Constraint(model.j,rule=con3_rule)

def obj_rule(model):
    return sum( model.x[i,j]*model.CargoProfit[i] for i in model.i for j in model.j)
model.OF = Objective(rule=obj_rule,sense=maximize)

In [10]:
instance = model.create_instance("Airplane Balance.dat")

In [11]:
opt = SolverFactory('glpk')
results = opt.solve(instance) # solves and updates instance

In [12]:
#instance.display()
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  result.solver.status)

feasible


In [13]:
for i in instance.i:
    for j in instance.j:
        print('X[',i,j,']=', round(value(instance.x[i,j]),2))
print('OF=',round(value(instance.OF),2))

for j in instance.j:
    print(j, sum(value(instance.x[i,j]) for i in instance.i))

X[ c1 f1 ]= 0.0
X[ c1 f2 ]= 0.0
X[ c1 f3 ]= 0.0
X[ c2 f1 ]= 10.0
X[ c2 f2 ]= 0.0
X[ c2 f3 ]= 5.0
X[ c3 f1 ]= 0.0
X[ c3 f2 ]= 12.95
X[ c3 f3 ]= 3.0
X[ c4 f1 ]= 0.0
X[ c4 f2 ]= 3.05
X[ c4 f3 ]= 0.0
OF= 12151.58
f1 10.0
f2 15.99999999999997
f3 8.0
